In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("match_samps.csv")
df2 = df.drop(["stats.name", "matchId", "assetsId", "matchType", "stats.deathType", "map", "stats.rank", "stats.playerId"], axis=1)
value_count = df2['gameMode'].value_counts()
df_value_counts = pd.DataFrame(value_count)
df_value_counts = df_value_counts.reset_index()
df_value_counts.columns = ['gameMode', 'num_games_played']
df3 = df2[df2['gameMode']=="solo"]
df3 = df3.drop(columns=["gameMode"])

def f(row):
    if row['stats.winPlace'] <= 5:
        val = 1
    else:
        val = 0

    return val
df3['top_five'] = df.apply(f, axis=1)

In [3]:
y = df3["top_five"]
target_names = ["UnRanked", "Top5"]
data = df3.drop(["top_five", "stats.winPlace"], axis=1)
data.head()

,gameDuration,stats.DBNOs,stats.assists,stats.boosts,stats.damageDealt,stats.headshotKills,stats.heals,stats.killPlace,stats.killStreaks,stats.kills,stats.longestKill,stats.revives,stats.rideDistance,stats.roadKills,stats.swimDistance,stats.teamKills,stats.timeSurvived,stats.vehicleDestroys,stats.walkDistance,stats.weaponsAcquired
286,2214,0.0,0.0,0.0,100.000000,0.0,0.0,37.0,1.0,1.0,0.00000,0.0,0.0000,0.0,0.0,0.0,1028.0,0.0,268.854000,0.0
287,2214,0.0,0.0,0.0,0.000000,0.0,0.0,90.0,0.0,0.0,0.00000,0.0,0.0000,0.0,0.0,0.0,789.0,0.0,28.123253,0.0
288,2214,0.0,0.0,4.0,427.098080,1.0,0.0,4.0,1.0,4.0,191.64577,0.0,2323.3994,0.0,0.0,0.0,1229.0,0.0,1692.429000,4.0
289,2214,0.0,0.0,4.0,199.413650,0.0,0.0,28.0,1.0,1.0,0.00000,0.0,0.0000,0.0,0.0,0.0,1647.0,0.0,1688.476400,0.0
290,2214,0.0,0.0,0.0,25.199999,0.0,0.0,44.0,0.0,0.0,0.00000,0.0,4562.7593,0.0,0.0,0.0,1629.0,0.0,1043.243900,13.0


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, random_state=42)

In [5]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
print(grid.best_params_)

In [ ]:
predictions = grid.predict(X_test)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["Top5", "Unranked"]))

In [ ]:
score_df = pd.DataFrame(columns=["Algorithm","Accuracy"])
score_df = score_df.append(pd.Series(["Grid Search", grid.score(X_test, y_test), index=["Algorithm", "Accuracy"]), ignore_index=True)
score_df.to_csv('score2.csv', mode='a', header=False)
score_df                                    s
                                      